Ваше завдання створити візуалізації, котрі відповідають на наступні питання:

- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

Ви самостійно маєте обрати спосіб візуалізації. Ви також маєте написати короткий супровідний текст до кожної візуалізації, котрий пояснює, чому ви обрали саме цей спосіб презентації даних, які ще альтернативи ви розглядали, та чому зупинились саме на цьому варіанті (які його переваги та недоліки).

Джерело даних: Національна енергетична компанія "Укренерго"


In [69]:
import altair as alt
import pandas as pd
import numpy as np

In [70]:
## Finding day of year
import datetime as dt

In [71]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [72]:
df = pd.read_excel('./data/2014-2020.xlsx')
df = df.rename(columns={'Час/Дата': 'Date'})
df.head()

,Date,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


### Як змінювалась структура генерації електроенергії за роками?

In [73]:
sources = ["AES", "TEC", "VDE", "TES", "GES", "GAES_GEN"]
years_range = list(range(2014, 2020))
columns = ["year", "source", "energy", ]

df_sources = pd.DataFrame(columns=columns)
for y in years_range:
    year_mask = df['Date'].str.contains(str(y))
    means = list(df[year_mask][sources].mean())
    for s, m in zip(sources, means):
        df_temp = pd.DataFrame([[y, s, m]], columns=columns)
        df_sources = df_sources.append(df_temp, ignore_index=True)

df_sources.head()

,year,source,energy
0,2014,AES,10068.997489
1,2014,TEC,1676.339155
2,2014,VDE,183.377397
3,2014,TES,7831.721119
4,2014,GES,912.223059


In [74]:
alt.Chart(df_sources).mark_bar().encode(
    x=alt.X('year:O', stack='zero', title="Рік"),
    y=alt.Y('energy:Q', title="МВт"),
    color=alt.Color('source', title="Джерело Ен")
).properties(
    width = 500,
    height = 400,
    title='Зміна структури генерації електроенергії за роками',
)

alt.Chart(...)

### Опис

Я обрав цей вид візуалізації бо він наглядний, та його легко зрозуміти. Для покращення цього графіку можна подумати про деталізацією, так щоб можна було отримати дані в числах а не тільки наглядно

### Як залежить споживання електроенергії від дня року та години доби?

In [75]:
def extract_year(x):
    return int(x[-4:])

def extract_month(x):
    return int(x.split(".")[1])

def extract_day(x):
    return int(x.split(".")[0].split("-")[1])

def extract_hour(x):
    return int(x.split("-")[0])

def get_weekday(x):
    d_m_y = [int(elm) for elm in x.split("-")[1].split(".")]
    return dt.datetime(d_m_y[2], d_m_y[1], d_m_y[0]).weekday()

def get_day_of_year(x):
    d_m_y = [int(elm) for elm in x.split("-")[1].split(".")]
    dt_value = dt.datetime(d_m_y[2], d_m_y[1], d_m_y[0])

    return int(dt_value.strftime('%j'))

df_dates = df
df_dates["year"] = df_dates["Date"].apply(extract_year)
df_dates["month"] = df_dates["Date"].apply(extract_month)
df_dates["day"] = df_dates["Date"].apply(extract_day)
df_dates["hour"] = df_dates["Date"].apply(extract_hour)
df_dates["weekday"] = df_dates["Date"].apply(get_weekday)
df_dates["day_of_year"] = df_dates["Date"].apply(get_day_of_year)

df_dates.head()


,Date,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12,year,month,day,hour,weekday,day_of_year
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN,2020,12,31,24,3,366
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN,2020,12,31,23,3,366
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN,2020,12,31,22,3,366
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN,2020,12,31,21,3,366
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN,2020,12,31,20,3,366


In [76]:
df_dates = df_dates.rename(columns={'CONSUMPTION': 'consumption'})
columns = ["day_of_year", "hour", "avg_consumption"]

df_consumption = pd.DataFrame(columns=columns)
for h in range(1, 24):
    for d in range(1, 367):
        hour_mask = df_dates['hour'] == h
        day_mask = df_dates['day_of_year'] == d
        m = int(df_dates[hour_mask & day_mask]['consumption'].mean())
        df_temp = pd.DataFrame([[d, h, m]], columns=columns)
        df_consumption = df_consumption.append(df_temp, ignore_index=True)

print(df_consumption)


     day_of_year hour avg_consumption
0              1    1           18202
1              2    1           16632
2              3    1           17207
3              4    1           17479
4              5    1           17663
...          ...  ...             ...
8413         362   23           18778
8414         363   23           18783
8415         364   23           19122
8416         365   23           19232
8417         366   23           18561

[8418 rows x 3 columns]


In [77]:
alt.Chart(df_consumption).transform_filter(alt.FieldRangePredicate(field='hour', range=[1, 24])).mark_rect().encode(
    x = alt.X('hour:O'),
    y = alt.Y('day_of_year:N', sort = alt.Sort(order = 'descending')),
    color = alt.Color('avg_consumption:Q', scale = alt.Scale(scheme = 'blueorange'))
).properties(title="Залежність споживання електроенергії від дня року та години доби")

alt.Chart(...)

### Опис

Я обрав цей вид візуалізації, бо я бачив подібний графік в коді до цієї практичної та подумав що це досить інтуїтивно буде для такго виду даних, хіт мапа від синього до теплого оранджевого в цьому допомагає.

### Як змінюється генерація електроенергії з різних джерел впродовж доби?


In [78]:

columns = ["hour", "source", "energy"]

df_sources_hours = pd.DataFrame(columns=columns)
for h in range(1, 24):
    hour_mask = df['Date'].str.contains(f"^{h}-.*$", regex=True)
    means = list(df[hour_mask][sources].mean())
    for s, m in zip(sources, means):
        df_temp = pd.DataFrame([[h, s, m]], columns=columns)
        df_sources_hours = df_sources_hours.append(df_temp, ignore_index=True)

df_sources_hours.head()

,hour,source,energy
0,1,AES,9533.777865
1,1,TEC,1428.560813
2,1,VDE,164.178334
3,1,TES,4883.335549
4,1,GES,413.657020


In [79]:
alt.Chart(df_sources_hours).mark_bar().encode(
    x=alt.X('hour:O', stack='zero', title="Година доби"),
    y=alt.Y('energy:Q', title="МВт"),
    color=alt.Color('source', title="Джерело Ен")
).properties(
    width = 900,
    height = 600,
    title='Як змінюється генерація електроенергії з різних джерел впродовж доби?',
)

alt.Chart(...)

### Опис

Я обрав цей вид візуалізації, бо завдання досить подібне до першого, де я використав це й жеш вид візуалізації.

### Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

In [80]:
columns = ["month", "hour", "avg_consumption"]

df_consumption_by_month = pd.DataFrame(columns=columns)
for h in range(1, 24):
    for m in range(1, 13):
        hour_mask = df_dates['hour'] == h
        month_mask = df_dates['month'] == m
        consmption_mean = int(df_dates[hour_mask & month_mask]['consumption'].mean())
        df_temp = pd.DataFrame([[m, h, consmption_mean]], columns=columns)
        df_consumption_by_month = df_consumption_by_month.append(df_temp, ignore_index=True)

df_consumption_by_month.head()

,month,hour,avg_consumption
0,1,1,17855
1,2,1,17505
2,3,1,16270
3,4,1,14712
4,5,1,13518


In [81]:
alt.Chart(df_consumption_by_month).transform_filter(alt.FieldRangePredicate(field='hour', range=[1, 24])).mark_rect().encode(
    x = alt.X('hour:O'),
    y = alt.Y('month:N', sort = alt.Sort(order = 'descending')),
    color = alt.Color('avg_consumption:Q', scale = alt.Scale(scheme = 'blueorange'))
).properties(title="Як змінюється споживання електроенергії впродовж доби у розрізі місяців року")

alt.Chart(...)

### Опис

Я обрав цей вид візуалізації, бо використання хітмапи мені сподобалось, і тут воно теж підходить під умову. 

In [82]:
columns = ["year_season", "hour", "avg_consumption"]
seasons_of_year = {1: [12, 1 ,2], 2: [3, 4, 5], 3: [6, 7, 8], 4: [9, 10, 11]}

df_consumption_by_seasons = pd.DataFrame(columns=columns)
for h in range(1, 24):
    for year_season_indx, season in enumerate(["Winter", "Spring", "Summer", "Fall"], start=1):
        hour_mask = df_dates['hour'] == h
        year_season_mask = df_dates['month'].isin(seasons_of_year[year_season_indx])
        consmption_mean = int(df_dates[hour_mask & year_season_mask]['consumption'].mean())
        df_temp = pd.DataFrame([[season, h, consmption_mean]], columns=columns)
        df_consumption_by_seasons = df_consumption_by_seasons.append(df_temp, ignore_index=True)

df_consumption_by_seasons.head()

,year_season,hour,avg_consumption
0,Winter,1,17547
1,Spring,1,14835
2,Summer,1,13697
3,Fall,1,14731
4,Winter,2,17015


In [83]:
alt.Chart(df_consumption_by_seasons).transform_filter(alt.FieldRangePredicate(field='hour', range=[1, 24])).mark_rect().encode(
    x = alt.X('hour:O'),
    y = alt.Y('year_season:N', sort = alt.Sort(order = 'descending')),
    color = alt.Color('avg_consumption:Q', scale = alt.Scale(scheme = 'blueorange'))
).properties(title="Як змінюється споживання електроенергії впродовж доби у розрізі пори року")

alt.Chart(...)

### Опис

Тут теж хітмапа піходить до умови і наглядність мене задовільняє.

### Як змінюється споживання електроенергії впродовж тижня?

In [84]:
columns = ["weekday", "avg_consumption"]
days_of_week = ["Monday", "Tuesday","Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df_consumption_by_weekday = pd.DataFrame(columns=columns)

for week_indx, week_name in enumerate(days_of_week):
    weekday_mask = df_dates['weekday'] == week_indx
    consmption_mean = int(df_dates[weekday_mask]['consumption'].mean())
    df_temp = pd.DataFrame([[week_name, consmption_mean]], columns=columns)
    df_consumption_by_weekday = df_consumption_by_weekday.append(df_temp, ignore_index=True)

print(df_consumption_by_weekday)


     weekday avg_consumption
0     Monday           17168
1    Tuesday           17310
2  Wednesday           17384
3   Thursday           17396
4     Friday           17306
5   Saturday           16743
6     Sunday           16192


In [85]:
alt.Chart(df_consumption_by_weekday).mark_bar(size=20).encode(
    alt.X(
        "weekday:N",
        sort=days_of_week,
        title="Day of Week"
    ),
    alt.Y(
        "avg_consumption:Q",
        title="Consumption"
    )
).properties(height=400, width=800)

alt.Chart(...)

### Опис

Тут хітмапа втрачає її плюс в тому що вона має X і Y, тому для того я використав бар чарт